In [ ]:
import numpy as np
import os
from utils import modify_single_grib, truncate_grib_file
from cdo import Cdo
cdo = Cdo()
cdo.debug = True


def modify_value(field, var, new_value):
    """
    Modify the Z field in the GRIB file.
    """
    for v in var:
        if v in field.variables:
            print(f"Modifying variable {v} in the field")
            field[v].data = np.full(field[v].shape, new_value)
    return field

indir='/lus/h2resw01/hpcperm/ccpd/ECE4-DATA/oifs/TL63L31/19900101'
climate_indir='/lus/h2resw01/hpcperm/ccpd/ECE4-DATA/oifs/TL63L31/climate.v020'
#outdir='/lus/h2resw01/scratch/ccpd/OIFS-playground'
outdir='/lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/19900101'
climate_outdir='/lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020'


# modify climate file
cdo.splitname(input=os.path.join(climate_indir,'ICMCLECE4'), 
              output=os.path.join(climate_outdir,'ICMCLECE4_temp_'), 
              options="--eccodes -f grb2")

match_dict = {
    "al": 0.15,
    "aluvp": 0.06,
    "aluvd": 0.06,
    "alnip": 0.06,
    "alnid": 0.06,
    "lai_lv": 0.,
    "lai_hv": 0.,
}

for var, new_value in match_dict.items():
    modify_single_grib(
        inputfile=os.path.join(climate_outdir, f'ICMCLECE4_temp_{var}.grb'),
        outputfile=os.path.join(climate_outdir, f'ICMCLECE4_mod_{var}.grb'),
        variables=[var],
        spectral=False,
        myfunction=modify_value,
        new_value=new_value
    )
    os.remove(os.path.join(climate_outdir, f'ICMCLECE4_temp_{var}.grb'))

variables = list(match_dict.keys())
paths = [os.path.join(climate_outdir, f'ICMCLECE4_mod_{var}.grb') for var in variables]
if os.path.exists(os.path.join(climate_outdir, 'ICMCLECE4')):
    os.remove(os.path.join(climate_outdir, 'ICMCLECE4'))
cdo.mergetime(options="-L", input=paths, 
              output=os.path.join(climate_outdir, 'ICMCLECE4_almost'))
cdo.settaxis("9999-01-15,00:00:00,1month", input=os.path.join(climate_outdir, 'ICMCLECE4_almost'),
              output=os.path.join(climate_outdir, 'ICMCLECE4'), options="-a")
for path in paths:
    os.remove(path)

# erase all orography
modify_single_grib(
    inputfile=os.path.join(indir,'ICMSHECE4INIT'),
    outputfile=os.path.join(outdir,'ICMSHECE4INIT'),
    variables='z',
    spectral=True,
    myfunction=modify_value,
    new_value=0. 
)

# truncate spectral variables to first harmonic (mean value)
truncate_grib_file(
    inputfile=os.path.join(outdir,'ICMSHECE4INIT'),
    variables=['t','d','vo','lnsp'],
    outputfile=os.path.join(outdir,'ICMSHECE4INIT'),
)

# erase all subgrid orography
modify_single_grib(
    inputfile=os.path.join(indir,'ICMGGECE4INIT'),
    outputfile=os.path.join(outdir,'ICMGGECE4INIT'),
    variables=['sdor', 'anor', 'isor', 'slor', 'lsm'],
    spectral=False,
    myfunction=modify_value,
    new_value=0.  
)

# set humidity to 0
modify_single_grib(
    inputfile=os.path.join(indir,'ICMGGECE4INIUA'),
    outputfile=os.path.join(outdir,'ICMGGECE4INIUA'),
    variables='q',
    spectral=False,
    myfunction=modify_value,
    new_value=0.  
)



# DEBUG - start =============================================================
CALL  :cdo -O -s --eccodes -f grb2 -splitname /lus/h2resw01/hpcperm/ccpd/ECE4-DATA/oifs/TL63L31/climate.v020/ICMCLECE4 /lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/ICMCLECE4_temp_
STDOUT:
STDERR:
# DEBUG - end ===============================================================
RETURNCODE:0
Converting to netcdf file /lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/ICMCLECE4_temp_al.grb
Modifying GRIB file /lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/ICMCLECE4_temp_al.grb using function modify_value
Modifying variable al in the field
Converting back to GRIB file /lus/h2resw01/scratch/ccpd/cdoPynn6b56fb
Converting to netcdf file /lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/ICMCLECE4_temp_aluvp.grb
Modifying GRIB file /lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/ICMCLECE4_temp_aluvp.grb using function modify

In [18]:
import xarray as xr
cdo.setgridtype("regular", input=os.path.join(climate_outdir, "ICMCLECE4"), output=os.path.join(climate_outdir, "prova.nc"), options="-r -f nc --eccodes")
inputfile = os.path.join(climate_outdir, "prova.nc")
ds = xr.open_dataset(inputfile, decode_times=False, decode_cf=False, decode_timedelta=False)
ds.to_netcdf(os.path.join(climate_outdir, "new.nc"))
cdo.remapnn(inputfile, input=os.path.join(climate_outdir, "new.nc"), output=os.path.join(climate_outdir, "new.grb"), options="-r -f grb2 --eccodes")

# DEBUG - start =============================================================
CALL  :cdo -O -s -r -f nc --eccodes -setgridtype,regular /lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/ICMCLECE4 /lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/prova.nc
STDOUT:
STDERR:
# DEBUG - end ===============================================================
RETURNCODE:0
# DEBUG - start =============================================================
CALL  :cdo -O -s -r -f grb2 --eccodes -remapnn,/lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/prova.nc /lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/new.nc /lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/new.grb
STDOUT:
STDERR:
# DEBUG - end ===============================================================
RETURNCODE:0


'/lus/h2resw01/hpcperm/ccpd/ECE4-DATA-test/oifs/TL63L31/climate.v020/new.grb'